In [2]:
import pandas as pd
import duckdb
from pandasql import sqldf 

sqllite = lambda q: sqldf(q, globals())


In [3]:
purchase = pd.read_csv("../data/DataPart1/purchase.csv", encoding="UTF-8")
purchase = purchase.rename(columns={c: c.strip() for c in purchase.columns})
purchase["PurchaseDate"] = purchase["PurchaseDate"].apply(
    pd.to_datetime, format="%m/%d/%Y"
)
purchase.head(3)


,PurchaseID,ProductID,CustomerID,PurchasePrice,PurchaseDate
0,1,1,1,130,2016-02-06
1,2,2,2,97,2016-01-07
2,3,3,3,200,2016-08-08


In [4]:
product = pd.read_csv("../data/DataPart1/product.csv", encoding="UTF-8")
product = product.rename(columns={c: c.strip() for c in product.columns})
product.head(3)


,ProductID,ProductName,RecommendedPrice,Category
0,1,DVD,105,LivingRoom
1,2,Microwave,98,Kitchen
2,3,Monitor,200,Office


In [5]:
customers = pd.read_csv("../data/DataPart1/customer.csv", encoding="UTF-8")
customers = customers.rename(columns={c: c.strip() for c in customers.columns})
customers.head(3)

,CustomerID,FirstName,LastName,City,State,Zip
0,1,John,Miller,Asbury,NY,23433
1,2,Fred,Hammill,Basham,AK,85675
2,3,Stan,Mellish,Callahan,WY,38556


# Question 1
Return the CustomerID, FirstName, and LastName of the Customers who have not made a Purchase in the last 6 months of 2016 (July-December).


In [6]:
# Make sure PurchaseDate is in the format of `%Y-%m-%d`.

query = """
    SELECT
        c.CustomerID,
        c.FirstName,
        c.LastName,
        pur.PurchaseID
    FROM
        customers c
        LEFT JOIN (
            SELECT
                CustomerID,
                PurchaseID
            FROM
                purchase
            WHERE
                strftime('%Y-%m', PurchaseDate) BETWEEN "2016-07"
                AND "2016-12"
        ) pur ON c.CustomerID = pur.CustomerID
    WHERE
        pur.PurchaseID IS NULL
"""
sqllite(query)


,CustomerID,FirstName,LastName,PurchaseID
0,1,John,Miller,None
1,2,Fred,Hammill,None
2,4,Adrian,Caparzo,None
3,9,Billy,Costigan,None
4,10,Shelly,Sipes,None
5,11,Chirsty,Melton,None
6,12,Amanda,Owens,None
7,13,Brittany,Smits,None
8,18,Natalie,Woods,None
9,19,Wendy,Hilton,None


# Question 2
Return the ProductCategory and the average PurchasePrice for those customers who have purchased three or more unique products.

In [7]:
sqllite("""
    SELECT
        group_concat(pro.Category) AS ProductCategory,
        avg(pur.PurchasePrice) AS AveragePurchasePrice
    FROM
        product pro
        INNER JOIN purchase pur ON pro.ProductID = pur.ProductID
    GROUP BY
        pur.CustomerID
    HAVING
        count(DISTINCT pur.ProductID) >= 3
""")

,ProductCategory,AveragePurchasePrice
0,"Office,Office,LivingRoom",173.666667


# Question 3
Return ProductName with the highest total PurchasePrice for each ProductCategory.

In [8]:
sqllite("""
    SELECT
        row_id,
        Category,
        ProductName,
        PurchasePrice
    FROM
        (
            SELECT
                row_number() over(
                    PARTITION by Category
                    ORDER BY
                        PurchasePrice DESC
                ) AS row_id,
                *
            FROM
                product pro
                INNER JOIN purchase pur ON pro.ProductID = pur.ProductID
        ) tmp
    WHERE
        row_id = 1
""")

,row_id,Category,ProductName,PurchasePrice
0,1,Kitchen,Refrigerator,1100
1,1,LivingRoom,DVD,250
2,1,Office,Monitor,300
